<h1>Calculate TF-IDF using functions and broadcast variables.</h1>
<p>TF-IDF or text frequency, inverse document frequency, is a way of rating the importance of a word in a particular document.  It is used in search engines, text vectorizers, just to name a few.  As an FYI, this is built into the Spark ETL package, and detailed here: http://spark.apache.org/docs/latest/mllib-feature-extraction.html#tf-idf <br/><br/>

To do so, we'll revisit the text sorting example from before, but we'll introduce a few new concepts like leveraging functions, and using broadcast variables.
</p>

In [1]:
from sklearn.datasets import fetch_20newsgroups
rdd = sc.parallelize(fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes')).data)

n_docs = rdd.count()
print n_docs

11314


<h2>Find the document frequency</h2>
<p>Here, were going to do a modified document frequency.  To do so, we'll use a function that will get the unique document tokens for us.  We'll be introducing two concepts, using functions for custom maps/filters, and the idea of a broadcast variable.<br/><br/>
Using functions for maps is super helpful if you want to do a transformation the same way in multiple places, or if the transformation is more complex than something you want to write in a single line.<br/><br/>
Broadcast variables are a little bit different.  If you are opperating on a distributed system, the remote cores don't know about the local instances.  You can either pass a local variable to a function, but that variable pay need to be passed hundreds of times to each node depending on the size of the data.  To avoid this needless use of bandwidth, you can create a broadcast variable.  What this does is it sends the variable to all the nodes a single time, so that they can use them whenever you call them.  The only way they differ from the variable, is that you'll need to call the `value` object (you can see it in the 1st line of 'unique_doc_words'
</p>

In [2]:
#import nltk -> if you need to download the stopword corpa, here's how to call the interface
#nltk.download()

In [3]:
def unique_doc_words(doc, stop_words):
    l_words = filter(lambda x: x not in stop_words.value and x != '',
                        map(lambda word: word.strip(".,-;?").lower(), doc.split()))
    return list(set(l_words))

In [4]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
print list(stop)[:5]

bc_stop = sc.broadcast(stop)

[u'all', u'just', u'being', u'over', u'both']


In [5]:
rdd_term_docs = rdd.flatMap(lambda doc: unique_doc_words(doc, bc_stop)).cache()
rdd_term_docs.count()

851674

<h2>Now for the document frequency</h2>
<p>The rest of the document frequency will look very familiar. We will add a second function so that we are left with a dictionary that will give the IDF score in the form:
$$IDF(t,N_D) = log( (N_D + 1)/ (DF(t)+1))$$
<br/>
Where ``N_D`` is the total number of documents, ``t`` is the term, and ``DF(t)`` is the document frequence for a given term.  Note that since we use a log, if a term exists in every document, it's ``IDF`` value will be 0.
</p>

In [6]:
from math import log10

#Remember to create broadcast variables!
bc_n_docs = sc.broadcast(n_docs)

def idf(doc_freq, n_docs):
    return log10((1+n_docs.value)/(1+doc_freq))

In [7]:
df = rdd_term_docs.map(lambda word: (word, 1))\
            .reduceByKey(lambda a, b: a+b)\
            .map(lambda x: (x[0], idf(x[1], bc_n_docs)))\
            .collectAsMap()
            
print list(df.iteritems())[:5]

[(u'considered)', 3.752586178740409), (u'1.21)', 3.752586178740409), (u'959-7274', 3.752586178740409), (u'1,800', 3.752586178740409), (u'canada"', 3.752586178740409)]


<h2>Woo hoo!</h2>
<p>If you've gotten through these three notebooks, I hope I've piqued your interest.  Spark is a huge library, and with a large open sourced community, it's expanding and getting better all the time.  I will try to build up a library of examples, but if you have a specific topic that you want to look into.  If you'd like to contribute, let me know and I'll have a look at what you would like to add.</p>